## DeepScorev2を調査するnotebook

In [61]:
import json

In [62]:
#データの読み込み
train_json_file_path = "./ds2_dense/deepscores_train.json"
test_json_file_path = "./ds2_dense/deepscores_test.json"

with open(train_json_file_path) as file:
	deepscores_train_json = json.load(file)
_
with open(test_json_file_path) as file:
	deepscores_test_json = json.load(file)

print(type(deepscores_train_json))
print(type(deepscores_test_json))


<class 'dict'>
<class 'dict'>


In [63]:
def get_dict_hierarchy(d: dict, indent: int=0):
    """
    辞書の階層構造を文字列として出力する関数
    階層ごとに4タブでインデントされる。
    """
    hierarchy = ""
    for key, value in d.items():
        hierarchy += ' ' * indent + str(key) + "\n"
        if isinstance(value, dict):
            hierarchy += get_dict_hierarchy(value, indent + 4)# 4タブのインデントに設定
        elif not isinstance(value, dict):
            hierarchy = hierarchy[:-1] # 改行コードを除く
            hierarchy += ': ' + str(value) + "\n"
    return hierarchy

In [64]:
#jsonの階層構造を見やすくしたテキストを出力
deepscores_train_json_dict_hierarchy = get_dict_hierarchy(deepscores_train_json)
deepscores_test_json_dict_hierarchy = get_dict_hierarchy(deepscores_test_json)

with open("deepscores_test_json_dict_hierarchy.txt", "w") as file:
    file.write(deepscores_test_json_dict_hierarchy)

with open("deepscores_train_json_dict_hierarchy.txt", "w") as file:
    file.write(deepscores_train_json_dict_hierarchy)

In [65]:
#ラベルidをキー、ラベル名を値とする辞書を作成
label_dict = {}
for label,d in deepscores_train_json['categories'].items():
    if d['annotation_set']=='deepscores':
        label_dict[label] = d['name']

In [66]:
print(len(label_dict))
for label,label_name in label_dict.items():
    print(label,label_name)

136
1 brace
2 ledgerLine
3 repeatDot
4 segno
5 coda
6 clefG
7 clefCAlto
8 clefCTenor
9 clefF
10 clefUnpitchedPercussion
11 clef8
12 clef15
13 timeSig0
14 timeSig1
15 timeSig2
16 timeSig3
17 timeSig4
18 timeSig5
19 timeSig6
20 timeSig7
21 timeSig8
22 timeSig9
23 timeSigCommon
24 timeSigCutCommon
25 noteheadBlackOnLine
26 noteheadBlackOnLineSmall
27 noteheadBlackInSpace
28 noteheadBlackInSpaceSmall
29 noteheadHalfOnLine
30 noteheadHalfOnLineSmall
31 noteheadHalfInSpace
32 noteheadHalfInSpaceSmall
33 noteheadWholeOnLine
34 noteheadWholeOnLineSmall
35 noteheadWholeInSpace
36 noteheadWholeInSpaceSmall
37 noteheadDoubleWholeOnLine
38 noteheadDoubleWholeOnLineSmall
39 noteheadDoubleWholeInSpace
40 noteheadDoubleWholeInSpaceSmall
41 augmentationDot
42 stem
43 tremolo1
44 tremolo2
45 tremolo3
46 tremolo4
47 tremolo5
48 flag8thUp
49 flag8thUpSmall
50 flag16thUp
51 flag32ndUp
52 flag64thUp
53 flag128thUp
54 flag8thDown
55 flag8thDownSmall
56 flag16thDown
57 flag32ndDown
58 flag64thDown
59 flag128

In [101]:
#ラベル名をキー、ラベルidとバウディングボックスと画像idを持つ辞書を値とする辞書を作成
result = {}
for label,label_name in label_dict.items():
    for d in deepscores_train_json['annotations'].values():
        if d['cat_id'][0]==label:
            result[label_name]={'label_id':label, 'img_id':d['img_id'], 'a_bbox':d['a_bbox']}
            break

In [121]:
for k,v in result.items():
    print(k,v)
print(len(result))

brace {'label_id': '1', 'img_id': '1523', 'a_bbox': [69.0, 210.0, 89.0, 447.0]}
ledgerLine {'label_id': '2', 'img_id': '679', 'a_bbox': [1880.0, 561.0, 1911.0, 564.0]}
repeatDot {'label_id': '3', 'img_id': '1439', 'a_bbox': [1280.0, 573.0, 1286.0, 579.0]}
segno {'label_id': '4', 'img_id': '1439', 'a_bbox': [1246.0, 498.0, 1286.0, 537.0]}
coda {'label_id': '5', 'img_id': '679', 'a_bbox': [184.0, 1423.0, 221.0, 1469.0]}
clefG {'label_id': '6', 'img_id': '679', 'a_bbox': [129.0, 458.0, 169.0, 571.0]}
clefCAlto {'label_id': '7', 'img_id': '815', 'a_bbox': [1032.0, 2248.0, 1065.0, 2300.0]}
clefCTenor {'label_id': '8', 'img_id': '1072', 'a_bbox': [1774.0, 1824.0, 1814.0, 1896.0]}
clefF {'label_id': '9', 'img_id': '1523', 'a_bbox': [107.0, 377.0, 165.0, 452.0]}
clef8 {'label_id': '11', 'img_id': '679', 'a_bbox': [140.0, 572.0, 151.0, 587.0]}
clef15 {'label_id': '12', 'img_id': '1275', 'a_bbox': [113.0, 379.0, 138.0, 395.0]}
timeSig0 {'label_id': '13', 'img_id': '563', 'a_bbox': [264.0, 749.0,

In [103]:
#imagesで定義されている画像数とidリスト
img_id_list_images = []
for d in deepscores_train_json['images']:
    img_id_list_images.append(d['id'])
img_id_list_images.sort()
print(f'画像数: {len(img_id_list_images)}')
print(img_id_list_images)

画像数: 1362
[2, 3, 4, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18, 19, 20, 21, 22, 23, 24, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 53, 54, 55, 56, 57, 59, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 74, 75, 76, 77, 78, 79, 80, 81, 85, 86, 87, 88, 90, 91, 92, 93, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 107, 108, 109, 110, 111, 113, 114, 115, 116, 117, 118, 120, 121, 122, 124, 125, 126, 127, 129, 131, 132, 133, 134, 135, 136, 137, 138, 140, 141, 142, 143, 144, 145, 147, 148, 149, 150, 151, 152, 154, 155, 157, 158, 159, 160, 162, 164, 165, 167, 169, 170, 172, 173, 174, 176, 177, 178, 179, 180, 181, 182, 183, 186, 187, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 205, 206, 207, 208, 209, 210, 211, 213, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 244, 246, 247, 248, 250, 252, 253, 256, 257, 259, 260, 261, 262, 263, 264, 265, 266, 

In [123]:
#annotationsで登場する画像の数とidのリスト
img_id_list_annotations = []
id_old = 0
for d in deepscores_train_json['annotations'].values():
    if len(img_id_list_annotations)==0:
       id_old = d['img_id']
       img_id_list_annotations.append(int(id_old))
    else:
        if d['img_id'] != id_old:
            id_old = d['img_id']
            img_id_list_annotations.append(int(id_old))
img_id_list_annotations.sort()
print(f'画像数: {len(img_id_list_annotations)}')
print(img_id_list_annotations)

画像数: 1362
[2, 3, 4, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18, 19, 20, 21, 22, 23, 24, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 53, 54, 55, 56, 57, 59, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 74, 75, 76, 77, 78, 79, 80, 81, 85, 86, 87, 88, 90, 91, 92, 93, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 107, 108, 109, 110, 111, 113, 114, 115, 116, 117, 118, 120, 121, 122, 124, 125, 126, 127, 129, 131, 132, 133, 134, 135, 136, 137, 138, 140, 141, 142, 143, 144, 145, 147, 148, 149, 150, 151, 152, 154, 155, 157, 158, 159, 160, 162, 164, 165, 167, 169, 170, 172, 173, 174, 176, 177, 178, 179, 180, 181, 182, 183, 186, 187, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 205, 206, 207, 208, 209, 210, 211, 213, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 244, 246, 247, 248, 250, 252, 253, 256, 257, 259, 260, 261, 262, 263, 264, 265, 266, 

In [105]:
set(img_id_list_images)==set(img_id_list_annotations)

True

In [108]:
img_id_list_result = []
for label_name, label_info in result.items():
    img_id_list_result.append(int(label_info['img_id']))
img_id_list_result.sort()
print(f'画像数: {len(img_id_list_result)}')
print(img_id_list_result)

画像数: 115
[143, 151, 285, 348, 376, 376, 380, 407, 417, 487, 498, 498, 498, 529, 563, 563, 583, 676, 679, 679, 679, 679, 679, 679, 679, 679, 679, 679, 679, 679, 679, 679, 679, 679, 679, 679, 679, 679, 679, 679, 679, 679, 679, 815, 815, 815, 815, 828, 871, 883, 992, 993, 1009, 1029, 1044, 1044, 1069, 1069, 1072, 1072, 1072, 1072, 1072, 1072, 1072, 1072, 1072, 1072, 1072, 1076, 1089, 1089, 1102, 1126, 1160, 1208, 1211, 1211, 1211, 1211, 1211, 1213, 1256, 1275, 1275, 1275, 1275, 1275, 1294, 1294, 1373, 1373, 1373, 1373, 1377, 1377, 1377, 1417, 1439, 1439, 1439, 1439, 1474, 1474, 1523, 1523, 1523, 1523, 1523, 1523, 1531, 1531, 1535, 1577, 1599]


In [117]:
from PIL import Image

hosei = [-5,-5,5,5]

for label_name, label_info in result.items():
    img_id = int(label_info['img_id'])
    for d in deepscores_train_json['images']:
        if d['id']==img_id:
            img_filename = d['filename']
            img_filepath = f'./ds2_dense/images/{img_filename}'
            save_path = f'./a_bbox_img/{label_name}.png'
            a_bbox = [x+y for x,y in zip(label_info['a_bbox'],hosei)]
            print(d['id'],a_bbox) 
            with Image.open(img_filepath) as img:
                cropped_img = img.crop(a_bbox)
                cropped_img.save(save_path)


1523 [64.0, 205.0, 94.0, 452.0]
679 [1875.0, 556.0, 1916.0, 569.0]
1439 [1275.0, 568.0, 1291.0, 584.0]
1439 [1241.0, 493.0, 1291.0, 542.0]
679 [179.0, 1418.0, 226.0, 1474.0]
679 [124.0, 453.0, 174.0, 576.0]
815 [1027.0, 2243.0, 1070.0, 2305.0]
1072 [1769.0, 1819.0, 1819.0, 1901.0]
1523 [102.0, 372.0, 170.0, 457.0]
679 [135.0, 567.0, 156.0, 592.0]
1275 [108.0, 374.0, 143.0, 400.0]
563 [259.0, 744.0, 292.0, 786.0]
1102 [2775.0, 2995.0, 2804.0, 3037.0]
1373 [1829.0, 144.0, 1858.0, 192.0]
1377 [1830.0, 597.0, 1860.0, 644.0]
1439 [334.0, 349.0, 370.0, 391.0]
993 [617.0, 668.0, 649.0, 710.0]
1377 [390.0, 1182.0, 420.0, 1229.0]
883 [1536.0, 1148.0, 1569.0, 1190.0]
1377 [1830.0, 636.0, 1861.0, 683.0]
407 [2274.0, 1013.0, 2306.0, 1055.0]
380 [314.0, 467.0, 353.0, 513.0]
417 [2065.0, 410.0, 2101.0, 471.0]
679 [1668.0, 142.0, 1698.0, 169.0]
679 [1631.0, 151.0, 1661.0, 177.0]
679 [1256.0, 857.0, 1285.0, 884.0]
679 [583.0, 134.0, 612.0, 161.0]
1523 [1363.0, 669.0, 1402.0, 697.0]
1523 [1597.0, 221.0

In [118]:
def create_image_grid(imgs_pathlist, grid_size, img_size, save_path):
    """
    Create a grid of imgs_pathlist and save as a single image.

    :param imgs_pathlist: List of image paths.
    :param grid_size: Tuple of (columns, rows) for the grid.
    :param img_size: Tuple of (width, height) for each image.
    :param save_path: Path to save the final image.
    """
    cols, rows = grid_size
    width, height = img_size
    grid_image = Image.new('RGB', (cols * width, rows * height))

    for index, img_path in enumerate(imgs_pathlist):
        if index >= cols * rows:  # If more imgs_pathlist than grid spaces, stop.
            break
        img = Image.open(img_path)
        img = img.resize(img_size)
        grid_image.paste(img, ((index % cols) * width, (index // cols) * height))

    grid_image.save(save_path)

In [139]:
from PIL import Image, ImageDraw, ImageFont
import os

def create_image_grid_with_text(images, grid_size, image_size, save_path):
    """
    Create a grid of images with file names printed on them, and save as a single image.

    :param images: List of image paths.
    :param grid_size: Tuple of (columns, rows) for the grid.
    :param image_size: Tuple of (width, height) for each image.
    :param save_path: Path to save the final image.
    """
    cols, rows = grid_size
    width, height = image_size
    grid_image = Image.new('RGB', (cols * width, rows * height))
    # Use default font
    font = ImageFont.load_default()

    for index, img_path in enumerate(images):
        if index >= cols * rows:
            break
        img = Image.open(img_path)
        img = img.resize(image_size)
        draw = ImageDraw.Draw(img)

        # Extract filename without extension
        filename = os.path.splitext(os.path.basename(img_path))[0]

        # Calculate text position (at the bottom center of each image)
        # Hardcoded text size
        text_size = (50, 10)  # Width and height of the text box
        text_x = (width - text_size[0]) // 2
        text_y = height - text_size[1] - 5  # 5 pixels from the bottom

        # Add text to image
        draw.text((text_x, text_y), filename, font=font, fill=(255, 50, 50))

        # Paste image into the grid
        grid_image.paste(img, ((index % cols) * width, (index // cols) * height))

    grid_image.save(save_path)

# Example usage
# Assuming we have a list of 136 image paths in 'image_paths'
# Set the grid size (11 columns, 12 rows)
# Set each image size (e.g., 100x100 pixels)
# Specify the path to save the final image

# image_paths = ['path/to/image1.jpg', 'path/to/image2.jpg', ..., 'path/to/image136.jpg']
# create_image_grid_with_text(image_paths, (11, 12), (100, 100), 'path/to/save/final_image.jpg')


In [140]:
import glob
label_img_path_list = glob.glob('./a_bbox_img/*.png')
grid_size = (12,12)
image_size = (100,100)
save_path = './all_label.png'
create_image_grid_with_text(label_img_path_list, grid_size, image_size, save_path)

In [125]:
# field_number_and_decimal_point_position = [[3,9],[1,2]]
# packed_decimal_field_number_list = [l[0] for l in field_number_and_decimal_point_position]
# print(packed_decimal_field_number_list)

[3, 1]
